In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local', appName="Assignment1_E1")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 16:08:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#hdfs dfs -mkdir -p data
#hdfs dfs -put data/small_conditions.csv data/

In [34]:
SUPPORT_THRESHOLD = 50
data = sc.textFile("data/small_conditions.csv")
header = data.first() #extract header

In [35]:
# START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
# PATIENT is the patient identifier
# CODE is a condition identifier 
# DESCRIPTION is the name of the condition

In [36]:
# Reorganizing the data to a list of baskets (lists)
# and removing the header row
item_baskets = data.filter(lambda row: row != header)\
    .map(lambda line: line.split(","))

#item_baskets.take(10)

In [48]:
# Apriori Phase 1

# Flat listing all the baskets 
items = item_baskets.flatMap(lambda x: x)

# Mapping -> create pairs (item, 1)
itemPairs = items.map(lambda item: (item, 1))

# Reducing
itemCounts = itemPairs.reduceByKey(lambda a, b: a + b)

# Keeping only the ones above the support threshold
freqItemCounts = itemCounts.filter(lambda item: item[1] >= SUPPORT_THRESHOLD)

#items.take(10)
#itemPairs.take(10)
#itemCounts.take(10)

# Taking the 10 most frequent itemsets for k = 1
freqItemCounts.takeOrdered(10, key=lambda x: -x[1])

[('', 783),
 ('444814009', 320),
 ('Viral sinusitis (disorder)', 320),
 ('195662009', 193),
 ('Acute viral pharyngitis (disorder)', 193),
 ('10509002', 143),
 ('Acute bronchitis (disorder)', 143),
 ('72892002', 142),
 ('Normal pregnancy', 142),
 ('162864005', 90),
 ('Body mass index 30+ - obesity (finding)', 90),
 ('271737000', 81),
 ('Anemia (disorder)', 81),
 ('15777000', 76),
 ('Prediabetes', 76),
 ('59621000', 62),
 ('Hypertension', 62),
 ('40055000', 53),
 ('Chronic sinusitis (disorder)', 53),
 ('43878008', 52)]

In [50]:
# Creating the frequent items table
freqItemTable = freqItemCounts.map(lambda x: x[0])

#freqItemTable.take(10)

['444814009',
 'Viral sinusitis (disorder)',
 '10509002',
 'Acute bronchitis (disorder)',
 '',
 '195662009',
 'Acute viral pharyngitis (disorder)',
 '72892002',
 'Normal pregnancy',
 '162864005']

In [65]:
# Auxiliary structures

# Combinations of pairs of frequent items without repetition 
pairs = freqItemTable.cartesian(freqItemTable) \
                    .filter(lambda x: x[0] != x[1]) \
                    .map(lambda x: tuple(sorted((x[0], x[1]))) ) \
                    .distinct()
    


pairs.collect()

[('444814009', 'Viral sinusitis (disorder)'),
 ('10509002', '444814009'),
 ('444814009', 'Acute bronchitis (disorder)'),
 ('', '444814009'),
 ('195662009', '444814009'),
 ('444814009', 'Acute viral pharyngitis (disorder)'),
 ('444814009', '72892002'),
 ('444814009', 'Normal pregnancy'),
 ('162864005', '444814009'),
 ('444814009', 'Body mass index 30+ - obesity (finding)'),
 ('40055000', '444814009'),
 ('444814009', 'Chronic sinusitis (disorder)'),
 ('444814009', '59621000'),
 ('444814009', 'Hypertension'),
 ('15777000', '444814009'),
 ('444814009', 'Prediabetes'),
 ('43878008', '444814009'),
 ('444814009', 'Streptococcal sore throat (disorder)'),
 ('271737000', '444814009'),
 ('444814009', 'Anemia (disorder)'),
 ('10509002', 'Viral sinusitis (disorder)'),
 ('Acute bronchitis (disorder)', 'Viral sinusitis (disorder)'),
 ('', 'Viral sinusitis (disorder)'),
 ('195662009', 'Viral sinusitis (disorder)'),
 ('Acute viral pharyngitis (disorder)', 'Viral sinusitis (disorder)'),
 ('10509002', 'A

In [70]:
for basket in item_baskets:
    basket = sc.parallelize(basket)
    basket_pairs = basket.cartesian(basket)
    basket_pairs.collect()

TypeError: 'PipelinedRDD' object is not iterable

In [ ]:
# Phase 2 ...
for basket in item_baskets:
    for item_1 in basket:
        if item_1 not in freqItemTable: 
            continue
        for item_2 in basket:
            if item_2 in freqItemTable:
                
            
        
